In [1]:
%matplotlib inline

import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pd. set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 0)

In [ ]:
alphavantage_csv_path = "/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL.csv"

In [ ]:
df = pd.read_csv(alphavantage_csv_path, low_memory=False)
df.sort_values(by=["date"], ascending=True, inplace=True)

df = df[df["date"]>='2010-01-01']

df["date"] = pd.to_datetime(df['date'])

df.set_index("date", inplace=True)

In [ ]:
df.shape[0]

3064

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3064 entries, 2010-01-04 to 2022-03-04
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   1. open               3064 non-null   float64
 1   2. high               3064 non-null   float64
 2   3. low                3064 non-null   float64
 3   4. close              3064 non-null   float64
 4   5. adjusted close     3064 non-null   float64
 5   6. volume             3064 non-null   float64
 6   7. dividend amount    3064 non-null   float64
 7   8. split coefficient  3064 non-null   float64
dtypes: float64(8)
memory usage: 215.4 KB


In [ ]:
df.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
date,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0


In [ ]:
twint_csv_path = "/app/StockPricePredictions/data/twint/apple_finbert_twint_20100101_20220304.csv"

In [ ]:
df_twint = pd.read_csv(twint_csv_path, low_memory=False, lineterminator='\n')

In [ ]:
df_twint.id.is_unique

True

In [ ]:
df_twint['date'] = pd.to_datetime(df_twint['date'])

In [70]:
df_twint = df_twint[[ "id", "date", "created_at", "tweet", "hashtags", "cashtags", "day", "hour", "nlikes", "nretweets"]]

In [71]:
df_twint.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41056 entries, 0 to 41055
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          41056 non-null  int64         
 1   date        41056 non-null  datetime64[ns]
 2   created_at  41056 non-null  float64       
 3   tweet       41056 non-null  object        
 4   hashtags    41056 non-null  object        
 5   cashtags    41056 non-null  object        
 6   day         41056 non-null  int64         
 7   hour        41056 non-null  int64         
 8   nlikes      41056 non-null  int64         
 9   nretweets   41056 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(3)
memory usage: 3.1+ MB


In [72]:
df_twint.head()

,id,date,created_at,tweet,hashtags,cashtags,day,hour,nlikes,nretweets
0,7273036289,2010-01-01,1.262365e+12,Apple 2.0: Tablet: Big iPhone or thin MacBook? http://bit.ly/5ubYQM $AAPL,[],['aapl'],5,16,2,7
1,7382848163,2010-01-04,1.262647e+12,"Apple 2.0: Apple tablet to ship in March, sources say http://bit.ly/6tiTqx $AAPL",[],['aapl'],1,23,1,6
2,7380526126,2010-01-04,1.262643e+12,"Apple Tablet Will Be 10-11 Inches, Ships In March, Says WSJ $AAPL by @jwyarow http://bit.ly/8jkW2K",[],['aapl'],1,22,4,9
3,7377981409,2010-01-04,1.262637e+12,CHART OF THE DAY: Android Taking Wind Out Of iPhone's Sails $GOOG $AAPL $RIMM http://bit.ly/8ghFFJ,[],"['goog', 'aapl', 'rimm']",1,20,11,7
4,7350195565,2010-01-04,1.262568e+12,Apple 2.0: How many iPhones did Apple sell? http://bit.ly/7XdUne $AAPL,[],['aapl'],1,1,6,10


In [74]:
def clean_tweet(df):
    # replace URLs with a whitespace
    df['tweet'] = df['tweet'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')
    
    return df

In [76]:
df_twint = clean_tweet(df_twint)

<ipython-input-74-8f87bdb64681>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tweet'] = df['tweet'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ')


In [77]:
df_twint.head()

,id,date,created_at,tweet,hashtags,cashtags,day,hour,nlikes,nretweets
0,7273036289,2010-01-01,1.262365e+12,Apple 2.0: Tablet: Big iPhone or thin MacBook? $AAPL,[],['aapl'],5,16,2,7
1,7382848163,2010-01-04,1.262647e+12,"Apple 2.0: Apple tablet to ship in March, sources say $AAPL",[],['aapl'],1,23,1,6
2,7380526126,2010-01-04,1.262643e+12,"Apple Tablet Will Be 10-11 Inches, Ships In March, Says WSJ $AAPL by @jwyarow",[],['aapl'],1,22,4,9
3,7377981409,2010-01-04,1.262637e+12,CHART OF THE DAY: Android Taking Wind Out Of iPhone's Sails $GOOG $AAPL $RIMM,[],"['goog', 'aapl', 'rimm']",1,20,11,7
4,7350195565,2010-01-04,1.262568e+12,Apple 2.0: How many iPhones did Apple sell? $AAPL,[],['aapl'],1,1,6,10


In [54]:
idx = 3

### Vader Sentiment

In [78]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
sentence = df_twint["tweet"][idx] 
vs = analyzer.polarity_scores(sentence)
print("{:-<65} {}".format(sentence, str(vs)))

CHART OF THE DAY: Android Taking Wind Out Of iPhone's Sails $GOOG $AAPL $RIMM    {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}


In [79]:
def getSIA(text):
    sia = SentimentIntensityAnalyzer()
    sentiment = sia.polarity_scores(text)
    return sentiment

negative = []
neutral = []
positive = []
compound = []

for i in range(0, len(df_twint["tweet"])):
    sia = getSIA(df_twint["tweet"][i])
    negative.append(sia["neg"])
    neutral.append(sia["neu"])
    positive.append(sia["pos"])
    compound.append(sia["compound"])

df_twint["negative"] = negative
df_twint["neutral"] = neutral
df_twint["positive"] = positive
df_twint["compound"] = compound

In [87]:
df_twint.set_index("date", inplace=True)

In [88]:
df_twint.head()

,id,created_at,tweet,hashtags,cashtags,day,hour,nlikes,nretweets,negative,neutral,positive,compound
date,,,,,,,,,,,,,
2010-01-01,7273036289,1.262365e+12,Apple 2.0: Tablet: Big iPhone or thin MacBook? $AAPL,[],['aapl'],5,16,2,7,0.0,1.0,0.0,0.0
2010-01-04,7382848163,1.262647e+12,"Apple 2.0: Apple tablet to ship in March, sources say $AAPL",[],['aapl'],1,23,1,6,0.0,1.0,0.0,0.0
2010-01-04,7380526126,1.262643e+12,"Apple Tablet Will Be 10-11 Inches, Ships In March, Says WSJ $AAPL by @jwyarow",[],['aapl'],1,22,4,9,0.0,1.0,0.0,0.0
2010-01-04,7377981409,1.262637e+12,CHART OF THE DAY: Android Taking Wind Out Of iPhone's Sails $GOOG $AAPL $RIMM,[],"['goog', 'aapl', 'rimm']",1,20,11,7,0.0,1.0,0.0,0.0
2010-01-04,7350195565,1.262568e+12,Apple 2.0: How many iPhones did Apple sell? $AAPL,[],['aapl'],1,1,6,10,0.0,1.0,0.0,0.0


In [89]:
df_twint_duplicated_index = df_twint[df_twint.index.duplicated(keep=False)]

In [95]:
df_twint_duplicated_index.shape[0]

40779

In [92]:
df_twint_nonduplicated_index = df_twint[~df_twint.index.duplicated(keep=False)]

In [94]:
df_twint_nonduplicated_index.shape[0]

277

In [103]:
# df_twint.groupby('date')['negative'].mean()

df_twint_groupby = df_twint.groupby("date").agg(
     negative = ("negative", "mean"),
     nuetral = ("neutral", "mean"),
     positive = ("positive", "mean"),
     compound = ("compound", "mean"),
     nlikes = ("nlikes", "sum"),
     nretweets = ("nretweets", "sum"),
     )


In [104]:
df_twint_groupby

,negative,nuetral,positive,compound,nlikes,nretweets
date,,,,,,
2010-01-01,0.000000,1.000000,0.000000,0.000000,2,7
2010-01-04,0.000000,1.000000,0.000000,0.000000,22,32
2010-01-05,0.100000,0.900000,0.000000,-0.229400,5,11
2010-01-06,0.154000,0.846000,0.000000,-0.476700,4,6
2010-01-07,0.000000,0.875500,0.124500,0.371500,12,13
...,...,...,...,...,...,...
2022-02-28,0.045889,0.892222,0.061833,0.138461,3813,442
2022-03-01,0.044458,0.939958,0.015583,-0.085358,4824,684
2022-03-02,0.044267,0.895467,0.060400,0.060020,4161,470


In [105]:
df_merge = pd.merge(df, df_twint_groupby, how="left", left_index=True, right_index=True)

In [108]:
# df.index
# df_text.index
df_merge[df_merge.compound.isnull()]
# df_merge.shape

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,negative,nuetral,positive,compound,nlikes,nretweets
date,,,,,,,,,,,,,,
2010-01-14,210.11,210.46,209.0200,209.430,6.403832,15460500.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-15,210.93,211.60,205.8700,205.930,6.296811,21216700.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-29,201.08,202.20,190.2500,192.063,5.872793,44498300.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-19,201.86,203.20,201.1100,201.670,6.166551,14838200.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-02-22,202.34,202.50,199.1900,200.416,6.128207,13948700.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-12,146.19,149.05,145.8400,148.890,148.484298,73779113.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-15,148.56,149.44,146.3700,149.030,148.623917,83281315.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-09-17,148.82,148.82,145.7600,146.060,145.662009,129868824.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
df.shape

(3064, 8)

In [111]:
df_twint_groupby.shape

(4175, 6)

In [114]:
df_merge.fillna(method="ffill", inplace=True)

In [118]:
df_merge[df_merge.nlikes.isnull()]

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,negative,nuetral,positive,compound,nlikes,nretweets
date,,,,,,,,,,,,,,


In [124]:
# df_merge.to_csv("/app/StockPricePredictions/data/alphavantage/time_series_daily_adjusted/AAPL_WITH_TWITTER_SA.csv")
df_merge.head()

,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient,negative,nuetral,positive,compound,nlikes,nretweets
date,,,,,,,,,,,,,,
2010-01-04,213.43,214.50,212.38,214.01,6.543876,17633200.0,0.0,1.0,0.000,1.0000,0.0000,0.0000,22.0,32.0
2010-01-05,214.60,215.59,213.25,214.38,6.555190,21496600.0,0.0,1.0,0.100,0.9000,0.0000,-0.2294,5.0,11.0
2010-01-06,214.38,215.23,210.75,210.97,6.450921,19720000.0,0.0,1.0,0.154,0.8460,0.0000,-0.4767,4.0,6.0
2010-01-07,211.75,212.00,209.05,210.58,6.438996,17040400.0,0.0,1.0,0.000,0.8755,0.1245,0.3715,12.0,13.0
2010-01-08,210.30,212.00,209.06,211.98,6.481804,15986100.0,0.0,1.0,0.000,1.0000,0.0000,0.0000,18.0,19.0


# Spacy TextBlob

In [119]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('en_core_web_md')
nlp.add_pipe('spacytextblob')
text = 'Nexus – Will It Change Telecom Industry Dynamics? '
doc = nlp(text)

In [120]:
print(doc._.blob.polarity)
# -0.125

print(doc._.blob.subjectivity)
# 0.9

0.0
0.0


# TextBlob

In [122]:
from textblob import TextBlob

def sentiment_analysis(tweet):

 def getSubjectivity(text):
   return TextBlob(text).sentiment.subjectivity
  
 #Create a function to get the polarity
 def getPolarity(text):
   return TextBlob(text).sentiment.polarity
  
 #Create two new columns ‘Subjectivity’ & ‘Polarity’

 subjectivity = getSubjectivity(tweet)
 polarity = getPolarity(tweet)

 def getAnalysis(score):
  if score < 0:
    return "Negative"
  elif score == 0:
    return "Neutral"
  else:
    return "Positive"

 # tweet [‘TextBlob_Analysis’] = tweet  [‘TextBlob_Polarity’].apply(getAnalysis )

 result = getAnalysis(polarity)

 print('Subjectivity:', subjectivity)
 print('Polarity:', polarity)
 print('Analysis:', result)
 # print('Sentiment:', sentiment)

idx = 5

sentiment_analysis(df_twint["tweet"][idx])

df_twint["tweet"][idx]

Subjectivity: 0.0
Polarity: 0.0
Analysis: Neutral


'Why Apple Bought Quattro Wireless And Is Getting Into Advertising $AAPL $GOOG by @fromedome   '